# Handle missing values in Categorical Features
In this notebook will be shown how to deal with categorical features with missing values.
It will be used the [Classified Ads for Cars](https://www.kaggle.com/mirosval/personal-cars-classifieds) dataset to predict the price of ADs through a simple model of Linear Regression. 

In order to show the various strategies and relevants pros / cons, we will focus on a particular categorical feature of this dataset, the **maker**, the name of the brand of cars (Toyota, Kia, Ford, Bmw, ...). 

We will cover the following techniques:
* **Replace** missing values with the *most frequent values*.
* **Delete** rows with null values.
* **Predict** values using a Classifier Algorithm (supervised or unsupervised)


## Steps of this notebook
* **Show Raw Data**: let's see how our dataset looks like.
* **Some Feature Engineering**: data cleaning and extraction of new features.
* **Dealing with missing values in Categorical Features**: we will deal missing values by comparing different techniques.
* **Conclusions**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Show Raw Data

In [ ]:
filename = "/kaggle/input/personal-cars-classifieds/all_anonymized_2015_11_2017_03.csv"

dtypes = {
    "maker": str, # brand name
    "model": str,
    "mileage": float, # km
    "manufacture_year": float,
    "engine_displacement": float,
    "engine_power": float,
    "body_type": str, # almost never present
    "color_slug": str, # also almost never present
    "stk_year": str,
    "transmission": str, # automatic or manual
    "door_count": str,
    "seat_count": str,
    "fuel_type": str, # gasoline or diesel
    "date_created": str, # when the ad was scraped
    "date_last_seen": str, # when the ad was last seen
    "price_eur": float} # list price converted to EUR

df = pd.read_csv(filename, dtype=dtypes)
print(f"Raw data has {df.shape[0]} rows, and {df.shape[1]} columns")
df.head()

It seems this dataset is chock full of missing values... here we can see how many and in which features

In [ ]:
# Tons of missing values
print(df.isna().sum())

Now let's look at the values of our data. Some are really unusual. A car with an engine power of 32k or 1 doesn't make much sense... or like a car with 1 million mileage, a price of a few cents, etc ...

In [ ]:
# Here we can see strage data, like the Min() and the Max() of some features
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

In [ ]:
df.info()

As previously said, missing values are not the only problem. This dataset presents various "unexpected" data. Let's take a quick look at, for example, **seat_count**, **door_count** or **stk_year**. Below here are shown. Due to certain values or their lack, certain features are unlikely to be useful for our purposes.

In [ ]:
df = df[df["price_eur"] != 1295.34]
df = df[df["price_eur"] > 150.00]

df_ = df.select_dtypes(exclude=['int', 'float'])
df_ = df_[['maker', 'seat_count', 'door_count', 'stk_year', 'transmission', 'fuel_type']]
for col in df_.columns:
    print("Col: ", col)
    print(df_[col].unique(), end='\n\n') # to print categories name only
    # print(df_[col].value_counts()) # to print count of every category

## Some Feature Engineering
Let's start to fill the numerous missing data of our dataset. The technique that I have decided to implement here is to calculate the average/mode value of the data grouped by some features (maker, model, manufacture_year), and then fill the missing values with the calculated data

In [ ]:
df_new = df.reset_index(drop=True).copy()

# Manufacture Year
min_year = 1970 # minimum threshold of the year. Lower values will be replaced with the average of manufacture_year
average_years = df_new.loc[df_new["manufacture_year"] >= min_year].\
groupby(["maker", "model"])["manufacture_year"].\
mean().round().rename("avg_manufacture_year").reset_index() # support column with the average year grouped by maker and model

df_filtered = df_new.merge(average_years, how="left", on=["maker", "model"]) # merge of the support column with the dataframe
df_filtered.loc[df_new["manufacture_year"] < min_year, "manufacture_year"] = np.nan # force values below the minimum threshold to nan
df_filtered["manufacture_year"] = df_filtered["manufacture_year"].fillna(df_filtered["avg_manufacture_year"]) # fill nan values with the average year
df_filtered = df_filtered[df_filtered['manufacture_year'].notna()].reset_index(drop=True) # delete of those rows that have not manufacture_year valorized yet

# Engine Power
min_engine_power = 50
average = df_filtered.loc[df_filtered["engine_power"] >= min_engine_power].\
groupby(["maker", "model", "manufacture_year"])["engine_power"].mean().round().rename("avg_engine_power").reset_index()

df_filtered2 = df_filtered.merge(average, how="left", on=["maker", "model", "manufacture_year"])
df_filtered2.loc[df_filtered["engine_power"] < min_engine_power, "engine_power"] = np.nan
df_filtered2["engine_power"] = df_filtered2["engine_power"].fillna(df_filtered2["avg_engine_power"])
df_filtered2 = df_filtered2[df_filtered2['engine_power'].notna()].reset_index(drop=True)

# Engine Displacement
min_engine_displacement = 600
average = df_filtered2.loc[df_filtered2["engine_displacement"] >= min_engine_displacement].\
groupby(["maker", "model"])["engine_displacement"].mean().round().rename("avg_engine_displacement").reset_index()

df_filtered3 = df_filtered2.merge(average, how="left", on=["maker", "model"])
df_filtered3.loc[df_filtered2["engine_displacement"] < min_engine_displacement, "engine_displacement"] = np.nan
df_filtered3["engine_displacement"] = df_filtered3["engine_displacement"].fillna(df_filtered3["avg_engine_displacement"])
df_filtered3 = df_filtered3[df_filtered3['engine_displacement'].notna()].reset_index(drop=True)

# Mileage
average = df_filtered3.groupby(["manufacture_year"])["mileage"].mean().\
round().rename("avg_mileage").reset_index() # support column with the average mileage with same manufacture_year

df_filtered4 = df_filtered3.merge(average, how="left", on=["manufacture_year"]) # merge of the support column with the dataframe
df_filtered4["mileage"] = df_filtered4["mileage"].fillna(df_filtered4["avg_mileage"]).reset_index(drop=True) # fill nan values with the average mileage

# Fuel Type
average = df_filtered4.loc[df_filtered4["fuel_type"].notna()].\
groupby(["maker","model"])["fuel_type"].agg(lambda x:x.value_counts().index[0]).\
rename("mode_fuel_type").reset_index() # support column with the most frequent value of fuel_type grouped by maker and model

df_filtered5 = df_filtered4.merge(average, how="left", on=["maker","model"]) # merge of the support column with the dataframe
df_filtered5["fuel_type"] = df_filtered5["fuel_type"].fillna(df_filtered5["mode_fuel_type"]) # fill nan values with the mode of fuel_type
df_filtered5 = df_filtered5[df_filtered5['fuel_type'].notna()].reset_index(drop=True) # delete of those rows that have not fuel_type valorized yet

df_filtered5 = pd.get_dummies(df_filtered5,columns=["fuel_type"]) # get dummies variables of this categorical feature 

df_cleaned = df_filtered5.copy()

Here we transform the dates features from string to datetime and then create a new feature called **ad_duration**, calculated as the distance in days between the two dates. This new feature will allow us to understand how tempting the AD is.

In [ ]:
# AD duration
df_cleaned['date_created'] = pd.to_datetime(df_cleaned['date_created']).dt.normalize()
df_cleaned['date_last_seen'] = pd.to_datetime(df_cleaned['date_last_seen']).dt.normalize()
df_cleaned['ad_duration'] = (df_cleaned['date_last_seen'] - df_cleaned['date_created']).dt.days # new feature: how much last the ad

As already mentioned, the **seat_count** feature presents very unexpected data. We then replace the missing values with the median and then create a new categorical feature

In [ ]:
features_to_fillna = ["seat_count"]
for feature in features_to_fillna:
    df_cleaned[feature] = pd.to_numeric(df_cleaned[feature], errors="coerce") # transform to numeric
    replace_with = df_cleaned[feature].median()                       # deduction of the median
    df_cleaned[feature].fillna(replace_with,inplace=True)             # replace null
    
df_cleaned["seat_str"] = np.select(
    condlist=[
        (df_cleaned["seat_count"] >= 0) & (df_cleaned["seat_count"] < 4),
        (df_cleaned["seat_count"] >= 4) & (df_cleaned["seat_count"] < 6),
        (df_cleaned["seat_count"] >= 6)],
    choicelist=[
        "small",
        "medium",
        "large",
        ],
    default="unknown")
df_cleaned = pd.get_dummies(df_cleaned,columns=["seat_str"])
df_cleaned = df_cleaned.drop(["seat_count"], axis=1)

With the **transmission** feature a few data is missing, so we can *Delete* rows with null values. With the remaining rows we can create two new binary columns starting from the initial categorical feature.

In [ ]:
df_cleaned = df_cleaned[df_cleaned['transmission'].notna()]
df_cleaned = pd.get_dummies(df_cleaned,columns=["transmission"])
#size_mapping = {"man":0,"auto":1}
#df_cleaned["transmission"] = df_cleaned["transmission"].map(size_mapping)

*Drop* support columns and unusable features

In [ ]:
# drop features
features_to_drop = ["model", "door_count", "body_type", "color_slug", "stk_year", "avg_manufacture_year", "date_created", "date_last_seen", "avg_engine_displacement","avg_engine_power","avg_mileage", "mode_fuel_type"]
df_cleaned = df_cleaned.drop(features_to_drop, axis="columns")

Now that we've cleaned all the features, with the exception of the **maker**, let's see how they look like

In [ ]:
# Missing values
print(df_cleaned.isna().sum())

In [ ]:
corr = df_cleaned.corr()
plt.subplots(figsize=(15,10))
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True, )

## Dealing with missing values in Categorical Features
Almost there! Now we just have to handle the **maker** feature, and we will do it with four different ways. Then, for each of them, we will create a simple model of Linear Regression for the prediction of the price.

* **1st Model**: Delete the entire column **maker**.
* **2nd Model**: Replace missing values with the *most frequent values*.
* **3rd Model**: Delete rows with null values.
* **4th Model**: Predict the missing values with the RandomForestClassifier.

In [ ]:
mse_list = []
r2_score_list = []

def remove_outliers(dataframe):
    '''
    return a dataframe without rows that are outliers in any column
    '''
    return dataframe\
    .loc[:, lambda df: df.std() > 0.04]\
    .loc[lambda df: (np.abs(stats.zscore(df)) < 3).all(axis=1)]

def plot_regression(Y_test, Y_pred):
    '''
    method that plot a linear regression line on a scatter plot
    '''
    x = Y_test
    y = Y_pred

    plt.xlabel("True label")
    plt.ylabel("Predicted label")
    plt.plot(x, y, 'o')

    m, b = np.polyfit(x, y, 1)

    plt.plot(x, m*x + b)

## Delete the entire column *Maker*
Our first basic try will be to create a model without the column **maker**. This "no-deal" practice is needed when the large amount of missing data threatens to invalidate the entire feature.

With this scenario we will probably have the worst scores on metrics. We will use them to compare the other methods.

In [ ]:
df_no_maker = df_cleaned.copy()

# drop of the maker feature 
df_no_maker = df_no_maker.drop("maker", axis="columns")

df_no_maker = remove_outliers(df_no_maker)

X = df_no_maker.drop("price_eur", axis=1).values
Y = np.log1p(df_no_maker["price_eur"].values)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.1, random_state=0)

ll = LinearRegression()
ll.fit(X_train, Y_train)
Y_pred = ll.predict(X_test)

mse_list.append(mean_squared_error(Y_test, Y_pred))
r2_score_list.append(r2_score(Y_test, Y_pred))

print("MSE: "+str(mean_squared_error(Y_test, Y_pred)))
print("R2 score: "+str(r2_score(Y_test, Y_pred)))

In [ ]:
plot_regression(Y_test, Y_pred)

This is our first try. Can we do better?

## Replace missing values with the most frequent data
A certainly more effective method is to assign the missing values with the most frequent data, the mode. But be careful that this could lead to an unbalanced dataset, in case the missing values are a considerable number.

In [ ]:
df_replace_mode = df_cleaned.copy()

replace_with = df_replace_mode["maker"].mode()             # deduction of the mode
df_replace_mode["maker"].fillna(replace_with,inplace=True) # replace null with the mode

df_replace_mode = pd.get_dummies(df_replace_mode,columns=["maker"])

df_replace_mode = remove_outliers(df_replace_mode)

X = df_replace_mode.drop("price_eur", axis=1).values
Y = np.log1p(df_replace_mode["price_eur"].values)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.1, random_state=0)

ll = LinearRegression()
ll.fit(X_train, Y_train)
Y_pred = ll.predict(X_test)

mse_list.append(mean_squared_error(Y_test, Y_pred))
r2_score_list.append(r2_score(Y_test, Y_pred))

print("MSE: "+str(mean_squared_error(Y_test, Y_pred)))
print("R2 score: "+str(r2_score(Y_test, Y_pred)))

In [ ]:
plot_regression(Y_test, Y_pred)

As expected, we got better scoring metrics than before

## Delete rows with null values
Another option is to delete rows that have null values. Absolutely not recommended if our dataset is very small, but easily feasible if there are missing only a few values or if we have a very large dataset


In [ ]:
df_del_rows = df_cleaned.copy()

# deleteing row with null maker
df_del_rows = df_del_rows[df_del_rows['maker'].notna()]
df_del_rows = pd.get_dummies(df_del_rows,columns=["maker"])

df_del_rows = remove_outliers(df_del_rows)

X = df_del_rows.drop("price_eur", axis=1).values
Y = np.log1p(df_del_rows["price_eur"].values)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.1, random_state=0)

ll = LinearRegression()
ll.fit(X_train, Y_train)
Y_pred = ll.predict(X_test)

mse_list.append(mean_squared_error(Y_test, Y_pred))
r2_score_list.append(r2_score(Y_test, Y_pred))

print("MSE: "+str(mean_squared_error(Y_test, Y_pred)))
print("R2 score: "+str(r2_score(Y_test, Y_pred)))

In [ ]:
plot_regression(Y_test, Y_pred)

## Predict the missing values with the RandomForestClassifier
The most interesting approach is for sure the predicting of missing values with a classification algorithm. This will give us the opportunity not to waste a good chunk of the dataset, and thus a large amount of information. If our predictions are accurate enough with this technique we should have the best metric scores.

Therefore:
- we have to split the dataset between rows with the column *maker* valorized, and rows with null values.
- the first dataframe will become the one on which we will create the classification model with the *maker* as target feature.
- use the model thus created to predict the missing values from the dataframe with null values
- merge the two dataframe into one
- training the linear regression model

Let's divide the dataset in two

In [ ]:
df_with_maker = df_cleaned[df_cleaned['maker'].notna()]
print("N. rows with maker not null:", df_with_maker.shape[0])

df_no_maker = df_cleaned[df_cleaned['maker'].isna()]
print("N. rows with maker null:", df_no_maker.shape[0])

Let's start by creating the classification model and take a look at its metrics

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

X = df_with_maker.drop("maker", axis=1).values
Y = df_with_maker["maker"].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

forest = RandomForestClassifier(n_estimators=45, max_depth=25, random_state=False, 
                                max_features=0.6, min_samples_leaf=3, n_jobs=-1)

forest.fit(X_train, Y_train)

y_pred_train = forest.predict(X_train)
y_pred = forest.predict(X_test)

y_pred_proba = forest.predict_proba(X_test)

accuracy_train = accuracy_score(Y_train, y_pred_train)
accuracy_test = accuracy_score(Y_test, y_pred)

print("ACCURACY: TRAIN=%.4f TEST=%.4f" % (accuracy_train,accuracy_test))
print("LOG LOSS: "+str(log_loss(Y_test, y_pred_proba)))

importances = forest.feature_importances_
indices = list(np.argsort(importances))[::-1]

plt.title("Feature importances")
plt.barh(range(len(indices)), importances[indices], color="g", align="center")
plt.yticks(range(len(indices)), df_with_maker.iloc[:, 1:].columns[indices])
# plt.ylim([-1, len(indices)])
plt.gca().invert_yaxis()

Without too much surprise the model tells us that **engine_displacement** and **engine_power** are the key features to determine the **maker**

Now that we have the model, let's fill the 2nd dataframe with the values of the prediction...

In [ ]:
df_no_maker = df_no_maker.drop('maker', axis=1)
prediction = forest.predict(df_no_maker)

df_no_maker.insert(0, 'maker', prediction)

...and then, merge the two dataframe togheter

In [ ]:
frames = [df_with_maker, df_no_maker]
df_final = pd.concat(frames)
print(df_final.shape[0])

Here's our final dataframe

In [ ]:
df_final.head()

Now that our final dataset is ready we can re-create the linear regression model and check if we have improved its metrics

In [ ]:
df_final = pd.get_dummies(df_final,columns=["maker"])

df_final = remove_outliers(df_final)

X = df_final.drop("price_eur", axis=1).values
Y = np.log1p(df_final["price_eur"].values)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.1, random_state=0)

ll = LinearRegression()
ll.fit(X_train, Y_train)
Y_pred = ll.predict(X_test)

mse_list.append(mean_squared_error(Y_test, Y_pred))
r2_score_list.append(r2_score(Y_test, Y_pred))

print("MSE: "+str(mean_squared_error(Y_test, Y_pred)))
print("R2 score: "+str(r2_score(Y_test, Y_pred)))

In [ ]:
plot_regression(Y_test, Y_pred)

Here's we can see the best metrics values thanks to the rows saved through the aforementioned dataset merge

## Conclusions
Our considerations are driven by metrics. Deleting the feature from the dataset should be our last resort. Replacing null data with the most frequent ones or deleting the rows can be a convenient solution in case we have few missing data or a very large dataset. On the contrary, in case we have a lot of missing values or a small dataset, a prediction/clustering could save us valuable information that would otherwise be lost.

In [ ]:
options_list = [
    "Delete the entire column Maker", 
    "Replace null values with the mode", 
    "Delete rows with null values", 
    "Predict the missing values"
]

df_metrics = pd.DataFrame({
    "": options_list,
    "MSE": mse_list,
    "R2_score": r2_score_list,
})
df_metrics.head()